In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import json
import seaborn as sns
from collections import Counter
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import gensim
import pandas as pd

In [2]:
import json
from collections import defaultdict
from tqdm import tqdm
import numpy as np

filename = 'yelp_academic_dataset_review.json'
target_user_ids = dict()
target_item_ids = dict()
reviews = []
i = 0
with open(filename, 'rt') as f:
        for line in tqdm(f):
            data = json.loads(line)

            reviews.append({
                key: data[key]
                for key in ['review_id', 'user_id', 'business_id', 'stars', "text"]
            })
            target_user_ids[data['user_id']] = True
            target_item_ids[data['business_id']] = True
            i+=1
            if i == 1000000:
                break
                
review_df = pd.DataFrame(reviews)
review_df["stars"] = review_df["stars"].astype(np.int32)
del reviews

999999it [00:17, 56740.74it/s] 


In [3]:
data = pd.read_csv (r'yelp_balanced.csv', index_col = None)

In [4]:
class MyCorpusSentsTagged:
    def __init__(self, data, test_data = False, include_stars = False):
        self.test_data = test_data
        self.data = data
        self.include_stars = include_stars

    def __iter__(self):
        for ix, row in enumerate(self.data.iloc):
            text = gensim.utils.simple_preprocess(row["text"])
            if self.test_data:
                yield text, row["stars"]
            else:
                if self.include_stars:
                    yield gensim.models.doc2vec.TaggedDocument(text, [ix, row["stars"]])
                else:
                    yield gensim.models.doc2vec.TaggedDocument(text, [ix])

    def __getitem__(self, key):
        row = self.data.iloc[key]
        text = gensim.utils.simple_preprocess(row["text"])
        if self.test_data:
            return text, row["stars"]
        else:
            if self.include_stars:
                return gensim.models.doc2vec.TaggedDocument(text, [row["user_id"], row["stars"]])
            else:
                return gensim.models.doc2vec.TaggedDocument(text, [row["user_id"]])
        
def infer(model, text):
    return model.infer_vector(text)

In [5]:
#corpus = MyCorpusSentsTagged(data)
#corpus[0]
corpus = MyCorpusSentsTagged(review_df, include_stars = True)

In [ ]:
#corpus = MyCorpusSentsTagged(data)
corpus = MyCorpusSentsTagged(review_df, include_stars = True)
model = gensim.models.doc2vec.Doc2Vec(vector_size = 300,
                                      min_count = 1,
                                      epochs = 8,
                                      workers = -1,
                                      min_alpha = .0025,
                                      max_vocab_size = None)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
x, y = [], []
#corp = MyCorpusSentsTagged(data, True)
corp = MyCorpusSentsTagged(review_df, True)
for doc, stars in corp:
    x.append(infer(model, doc))
    y.append(stars)
y_ = np.array(y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
enc.fit(y_.reshape(-1,1))
y = enc.transform(y_.reshape(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_, test_size=.2, random_state=42)

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=-1, C=1e5)
logreg.fit(x_train, y_train)
logreg.score(np.array(x_test), y_test)

In [ ]:
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

names = ["Decision Tree", "Neural Net", "OLS", "Linear SVM"]

classifiers = [    
    DecisionTreeClassifier(),                  
    MLPClassifier(alpha=1e-5, hidden_layer_sizes=(50, 5)),
    linear_model.LinearRegression(),
    SGDClassifier(loss='hinge',
                                      penalty='l2',
                                      alpha=30,
                                      max_iter=1000, 
                                      tol=1e-3,
                                      shuffle=True,
                                      verbose=1,
                                      n_jobs=-1,
                                      random_state=0,
                                      learning_rate='optimal',
                                      early_stopping=True,
                                      class_weight='balanced')
]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)
x_train, x_test, y_train_, y_test_ = train_test_split(x, y_, test_size=.2, random_state=42)


# iterate over classifiers
for name, clf in zip(names, classifiers):
    try:
        clf.fit(np.array(x_train), y_train)
        score = clf.score(np.array(x_test), y_test)
    except Exception as E:
        clf.fit(np.array(x_train), y_train_)
        score = clf.score(np.array(x_test), y_test_)
    print(name, score)

min_score = 0
min_val = 0
for val in np.arange(2, 30, 15):
    clf = KNeighborsClassifier(val)
    clf.fit(x_train, y_train)
    tmp = clf.score(np.array(x_test), y_test)
    if tmp > min_score:
        min_val = val
        min_score = tmp
print("%s Nearest Neighbors" % min_val, min_score)

Decision Tree 0.20096666666666665
Neural Net 0.0
OLS -0.002585289857950501


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 0.00, NNZs: 300, Bias: -0.688178, T: 108000, Avg. loss: 0.538661
Total training time: 0.18 seconds.
Norm: 0.00, NNZs: 300, Bias: 0.146964, T: 108000, Avg. loss: 0.990244
Total training time: 0.17 seconds.
Norm: 0.00, NNZs: 300, Bias: -0.699612, T: 108000, Avg. loss: 0.533123
Total training time: 0.17 seconds.
Norm: 0.00, NNZs: 300, Bias: -0.676928, T: 108000, Avg. loss: 0.546166
Total training time: 0.16 seconds.
Norm: 0.00, NNZs: 300, Bias: -0.683050, T: 108000, Avg. loss: 0.541663
Total training time: 0.17 seconds.
-- Epoch 2
-- Epoch 2
-- Epoch 2
-- Epoch 2
-- Epoch 2
Norm: 0.00, NNZs: 300, Bias: -0.702056, T: 216000, Avg. loss: 0.523673
Total training time: 0.51 seconds.
Norm: 0.00, NNZs: 300, Bias: -0.696950, T: 216000, Avg. loss: 0.526613
Total training time: 0.49 seconds.
Norm: 0.00, NNZs: 300, Bias: 0.133110, T: 216000, Avg. loss: 0.975225
Total training time: 0.54 seconds.
-- Epoch 3
Norm: 0.00, NNZs: 300, Bias: -0.7

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.4s finished


Linear SVM 0.2006


In [54]:
df = pd.read_csv("examples_combined.csv")
df.dropna(inplace = True)
df.astype({"contradictory" : "int32", "figurative" : "int32",
           "confusing" : "int32", "qualifier" : "int32", })
contradicting = df.loc[df["contradictory"] == 1]
confusing = df.loc[df["confusing"] == 1]
qualifier = df.loc[df["qualifier"] == 1]
figurative = df.loc[df["figurative"] == 1]
assert all(len (i) > 0 for i in [contradicting, confusing, qualifier, figurative])

In [74]:
x, y = [], []
for doc in contradicting.iloc:
    x.append(infer(model, [doc["text"]]))
    y.append(doc["stars"])
y = enc.transform(np.array(y).reshape(-1,1))
classifiers[3].predict(x)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [82]:
def stars(star):
    if star > 3: return 1
    if star == 3: return 0
    if star < 3: return -1

class MyCorpusSentsTagged:
    def __init__(self, data, test_data = False):
        self.test_data = test_data
        self.data = data

    def __iter__(self):
        for row in data.iloc:
            if self.test_data:
                yield row["text"], stars(row["stars"])
            else:
                yield gensim.models.doc2vec.TaggedDocument(row["text"], [stars(row["stars"])])

    def __getitem__(self, key):
        row = self.data.iloc[key]
        if self.test_data:
            return row["text"], stars(row["stars"])
        else:
            return gensim.models.doc2vec.TaggedDocument(row["text"], [stars(row["stars"]), row["user_id"]])

In [73]:
corpus = MyCorpusSentsTagged(data)
model = gensim.models.doc2vec.Doc2Vec(vector_size = 400,
                                      min_count=1,
                                      epochs = 10,
                                      workers = -1,
                                      max_vocab_size = None)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [84]:
x, y = [], []
corp = MyCorpusSentsTagged(data, True)
for doc, stars_ in corp:
    x.append(infer(model, [doc]))
    y.append(stars_)
y_ = np.array(y)

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
enc.fit(y_.reshape(-1,1))
y = enc.transform(y_.reshape(-1,1))

In [85]:
names = ["Decision Tree", "Neural Net", "OLS", "Linear SVM"]

classifiers = [    
    DecisionTreeClassifier(),                  
    MLPClassifier(alpha=1e-5, hidden_layer_sizes=(50, 5)),
    linear_model.LinearRegression(),
    SGDClassifier(loss='hinge',
                                      penalty='l2',
                                      alpha=30,
                                      max_iter=1000, 
                                      tol=1e-3,
                                      shuffle=True,
                                      verbose=1,
                                      n_jobs=-1,
                                      random_state=0,
                                      learning_rate='optimal',
                                      early_stopping=True,
                                      class_weight='balanced')
]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)
x_train, x_test, y_train_, y_test_ = train_test_split(x, y_, test_size=.2, random_state=42)


# iterate over classifiers
for name, clf in zip(names, classifiers):
    try:
        clf.fit(np.array(x_train), y_train)
        score = clf.score(np.array(x_test), y_test)
    except Exception as E:
        clf.fit(np.array(x_train), y_train_)
        score = clf.score(np.array(x_test), y_test_)
    print(name, score)

min_score = 0
min_val = 0
for val in np.arange(2, 30, 4):
    clf = KNeighborsClassifier(val)
    clf.fit(x_train, y_train)
    tmp = clf.score(np.array(x_test), y_test)
    if tmp > min_score:
        min_val = val
        min_score = tmp
print("%s Nearest Neighbors" % min_val, min_score)

Decision Tree 0.36013333333333336
Neural Net 0.0001
OLS -0.002739648801408793


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 0.00, NNZs: 400, Bias: 0.302567, T: 108000, Avg. loss: 0.955811
Total training time: 0.11 seconds.
Norm: 0.00, NNZs: 400, Bias: -0.571030, T: 108000, Avg. loss: 0.798267
Total training time: 0.11 seconds.
Norm: 0.00, NNZs: 400, Bias: -0.641580, T: 108000, Avg. loss: 0.562092
Total training time: 0.11 seconds.
-- Epoch 2
-- Epoch 2
-- Epoch 2
Norm: 0.00, NNZs: 400, Bias: -0.652339, T: 216000, Avg. loss: 0.550445
Total training time: 0.37 seconds.
Norm: 0.00, NNZs: 400, Bias: -0.577203, T: 216000, Avg. loss: 0.796011
Total training time: 0.38 seconds.
Norm: 0.00, NNZs: 400, Bias: 0.296429, T: 216000, Avg. loss: 0.953601
Total training time: 0.38 seconds.
-- Epoch 3
-- Epoch 3
-- Epoch 3
Norm: 0.00, NNZs: 400, Bias: -0.580813, T: 324000, Avg. loss: 0.795168
Total training time: 0.65 seconds.
Norm: 0.00, NNZs: 400, Bias: -0.658638, T: 324000, Avg. loss: 0.546056
Total training time: 0.65 seconds.
Norm: 0.00, NNZs: 400, Bias: 0.292837, T: 324000, Avg. 

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.7s finished


Linear SVM 0.3977
6 Nearest Neighbors 0.14186666666666667
